# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
import re
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import pandas as pd

# Collect gt annotations and predicted images

In [9]:
def extract_id(gt_annotation):
  match = re.search(r"_([0-9]+)\.png$", gt_annotation["filename"])

  if match:
      gt_id = match.group(1)
  
  return gt_id

def find_gt_annotation(gt_annotations,pred_image):
  gt_image = None
  # Returns the gt_annotation that matches the pred_image 
  for annotation in gt_annotations:
    if str(extract_id(annotation)) == str(pred_image["image_id"]):
      gt_image = annotation  
  return gt_image

def get_pred_image_boxes(pred_image):
  boxes = []
  i=1 #Just if there is more than 1 mouse
  for box in pred_image["boxes"]:
    height= abs(box[1]-box[3])
    width = abs(box[0]-box[2])
    boxes.append({'mouse'+str(i): 'boundingBox', 
          'height': height, 
          'width': width, 
          'x': width/2 + min(box[0],box[2]), 
          'y': height/2+min(box[1],box[3])})
    i+=1
  return boxes

# Calculate IoU (Intersection over Union) scores

In [10]:
def calculate_iou(boxA, boxB):
    # Get coordinates of intersection rectangle
    xA = max(boxA['x'], boxB['x'])
    yA = max(boxA['y'], boxB['y'])
    xB = min(boxA['x'] + boxA['width'], boxB['x'] + boxB['width'])
    yB = min(boxA['y'] + boxA['height'], boxB['y'] + boxB['height'])

    # Calculate area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Calculate area of both bounding boxes
    boxAArea = boxA['width'] * boxA['height']
    boxBArea = boxB['width'] * boxB['height']

    # Calculate IoU
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

# Match predicted tracks to ground truth tracks

In [11]:
def match_predictions_gt(alpha=0.5,results={}):
  # Match is when the iou>alpha
  alpha = 0.5
  TPTr = {k: v for k, v in results.items() if v["iou"] >= alpha}
  PrTrajs = dict(sorted(results.items(), key=lambda x: x[1]["iou"], reverse=True))
  # [print(sorted_results[item]) for item in results]
  # check last item in results because it is a list with None objects
  return TPTr,PrTrajs

# Calculate precision and recall values

In [12]:
def calc_prec_recall(PrTrajs,results,alpha=0.5):
  n = 1
  TPTrn = 0
  for item in PrTrajs:
    if PrTrajs[item]["iou"] > alpha:
      TPTrn += 1
    PrTrajs[item]["Precision"] = TPTrn/n
    PrTrajs[item]["Recall"] = TPTrn/len(results)
    n+=1
    # print(TPTrn,n)
  return PrTrajs

# Calculate MOTA

In [13]:
from collections import defaultdict
from scipy.optimize import linear_sum_assignment

def compute_mota(pred_boxes, gt_boxes, alpha=0.5):
    num_pred = len(pred_boxes)
    num_gt = len(gt_boxes)

    iou_matrix = np.zeros((num_pred, num_gt))

    for i, pred_box in enumerate(pred_boxes):
        for j, gt_box in enumerate(gt_boxes):
            iou_matrix[i, j] = calculate_iou(pred_box, gt_box)

    row_ind, col_ind = linear_sum_assignment(-iou_matrix)

    tp = 0
    for i, j in zip(row_ind, col_ind):
        if iou_matrix[i, j] >= alpha:
            tp += 1

    fn = num_gt - tp
    fp = num_pred - tp
    id_sw = 0  # Set this to 0 for single frame MOTA

    mota = 1 - (fn + fp + id_sw) / num_gt if num_gt > 0 else 0
    return mota

def calculate_mota(pred_path, gt_path, alpha=0.5):

    with open(pred_path, 'r') as f:
        # Load the JSON data into a dictionary
        pred_annotations = json.load(f)

    with open(gt_path, 'r') as f:
        # Load the JSON data into a dictionary
        gt_annotations = json.load(f)

    mota_sum = 0
    num_frames = 0

    for image in pred_annotations:
        gt_image = find_gt_annotation(gt_annotations, image)
        if gt_image != None:
            num_frames += 1
            pred_boxes = get_pred_image_boxes(image)
            gt_boxes = [item for item in gt_image["annotations"] if len(item) > 3]

            mota = compute_mota(pred_boxes, gt_boxes, alpha)
            mota_sum += mota

    mota_avg = mota_sum / num_frames
    print(f"The average MOTA score is: {mota_avg}")
    return mota_avg
   

In [14]:
def get_file(string):
  terms = string.split("/")[-3:]
  joined_terms = "/".join(terms)
  return joined_terms[:-5]

In [16]:
import numpy as np
path = '/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/'

directories = [dir for dir in os.listdir(path) if os.path.isdir(os.path.join(path, dir))]
pred_path = "/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/topview/alphapose-results.json"
evaluation = {}
for file in os.listdir(path+"/"+directories[2]):
  if "top" in file:
    gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/one_mouse_top.json"
    pred_path = path+directories[2]+"/"+file+"/alphapose-results.json"
    print(pred_path)
    evaluation[get_file(pred_path)] = calculate_mota(pred_path,gt_path)
  elif "side" in file:
    gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/one_mouse_side.json"
    pred_path = path+directories[2]+"/"+file+"/alphapose-results.json"
    print(pred_path)
    evaluation[get_file(pred_path)] = calculate_mota(pred_path,gt_path)
  elif "face" in file:
    gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/one_mouse_face.json"
    pred_path = path+directories[2]+"/"+file+"/alphapose-results.json"
    print(pred_path)
    evaluation[get_file(pred_path)] = calculate_mota(pred_path,gt_path)
    

/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/sideview/alphapose-results.json
The average MOTA score is: -0.8583247156153051
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/faceview_ckpt2/alphapose-results.json
The average MOTA score is: -1.0964912280701755
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/faceview/alphapose-results.json
The average MOTA score is: -1.6558005752636624
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/sideview_ckpt2/alphapose-results.json
The average MOTA score is: -1.0283842794759825
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/sideview_ckpt1/alphapose-results.json
The average MOTA score is: -0.17789165446559296
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_in_a_new_cage/faceview_ckpt1/alphapose-results.json
The average MOTA score is: -0.37462686567164

In [22]:
for directory in directories:
  for file in os.listdir(path+"/"+directory):
    if "top" in file:
      gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/two_mice_top.json"
      pred_path = path+directory+"/"+file+"/alphapose-results.json"
      print(pred_path)
      evaluation[get_file(pred_path)] = calculate_mota(pred_path,gt_path)
    elif "side" in file:
      gt_path = "//content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/two_mice_side.json"
      pred_path = path+directory+"/"+file+"/alphapose-results.json"
      print(pred_path)
      evaluation[get_file(pred_path)] = calculate_mota(pred_path,gt_path)
    elif "face" in file:
      gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/two_mice_face.json"
      pred_path = path+directory+"/"+file+"/alphapose-results.json"
      print(pred_path)
      evaluation[get_file(pred_path)] = calculate_mota(pred_path,gt_path)

/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/sideview_ckpt1/alphapose-results.json


KeyboardInterrupt: ignored

In [ ]:

# [print(evaluation) for evaluation in evaluation.items()]
path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/evaluation_MOTA.json"

evaluations_df = pd.DataFrame(evaluation.items(),columns = ["File","MOTA"])
with open(path, "w") as outfile:
    json.dump(evaluation, outfile)

<h2> TASK 5

In [26]:
from collections import defaultdict
from scipy.optimize import linear_sum_assignment

def compute_mota(pred_boxes, gt_boxes, alpha=0.5):
    num_pred = len(pred_boxes)
    num_gt = len(gt_boxes)

    iou_matrix = np.zeros((num_pred, num_gt))

    for i, pred_box in enumerate(pred_boxes):
        for j, gt_box in enumerate(gt_boxes):
            iou_matrix[i, j] = calculate_iou(pred_box, gt_box)

    row_ind, col_ind = linear_sum_assignment(-iou_matrix)

    tp = 0
    for i, j in zip(row_ind, col_ind):
        if iou_matrix[i, j] >= alpha:
            tp += 1

    fn = num_gt - tp
    fp = num_pred - tp
    id_sw = 0  # Set this to 0 for single frame MOTA

    mota = 1 - (fn + fp + id_sw) / num_gt if num_gt > 0 else 0
    return mota

def calculate_mota_test(pred_path, gt_path, alpha=0.5):

    with open(pred_path, 'r') as f:
        # Load the JSON data into a dictionary
        pred_annotations = json.load(f)

    with open(gt_path, 'r') as f:
        # Load the JSON data into a dictionary
        gt_annotations = json.load(f)

    mota_sum = 0
    num_frames = 0

    for image in pred_annotations:
        gt_image = find_gt_annotation(gt_annotations, image)
        if gt_image != None:
            num_frames += 1
            pred_boxes = get_pred_image_boxes(image)
            gt_boxes = [item for item in gt_image["annotations"] if len(item) > 3]

            mota = compute_mota(pred_boxes, pred_boxes, alpha)
            mota_sum += mota

    mota_avg = mota_sum / num_frames
    print(f"The average MOTA score is: {mota_avg}")
    return mota_avg

In [ ]:
for directory in directories:
  for file in os.listdir(path+"/"+directory):
    if "top" in file:
      gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/two_mice_top.json"
      pred_path = path+directory+"/"+file+"/alphapose-results.json"
      print(pred_path)
      evaluation[get_file(pred_path)] = calculate_mota_test(pred_path,gt_path)
    elif "side" in file:
      gt_path = "//content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/two_mice_side.json"
      pred_path = path+directory+"/"+file+"/alphapose-results.json"
      print(pred_path)
      evaluation[get_file(pred_path)] = calculate_mota_test(pred_path,gt_path)
    elif "face" in file:
      gt_path = "/content/drive/MyDrive/Master_Mouse_Project/Submission/Task4/two_mice_face.json"
      pred_path = path+directory+"/"+file+"/alphapose-results.json"
      print(pred_path)
      evaluation[get_file(pred_path)] = calculate_mota_test(pred_path,gt_path)

/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/sideview_ckpt1/alphapose-results.json
The average MOTA score is: 1.0
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/faceview_ckpt2/alphapose-results.json
The average MOTA score is: 1.0
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/faceview_ckpt1/alphapose-results.json
The average MOTA score is: 1.0
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/faceview/alphapose-results.json
The average MOTA score is: 1.0
/content/drive/MyDrive/Master_Mouse_Project/Data/track_result_/A_male_meet_with_the_other_cage_male/sideview/alphapose-results.json
